### Setup



In [ ]:
from platform import python_version

print(python_version())
#!nvidia-smi

import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter;
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)


### Reading Dataset

In [ ]:
data_cleaned = pd.read_csv('SQL_extracted datasets/Formatted_data/FINALlanghill_mir_extractready_revised_7pm_mir_aligned_20210711.csv', sep=',')
print(data_cleaned.shape) #Shape is 417657 rows and 18 columns

### Split Data - Basic split
80% Train, 10% Validation, 10% Test <br> Could also split by time (e.g. train 2013-2018, test 2018-2021) - Could randomize the data and split: most likely choice


In [ ]:
#Split data into 3 - train validation and test
#Once model trained, test the model using best interation of predictions
#For comparison of predicted vs observed
#Plots of accuracy
#Alternative split method which will randomly split the data
from sklearn.model_selection import train_test_split

test_size = 0.1
X = data_cleaned.iloc[:, 14:1074]
y = data_cleaned['weeklyave_FI']

X_test, X_train, y_test, y_train =train_test_split(X,y, train_size=0.1, random_state = 11)
#_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, train_size=0.8)
# check dimensions
print('X_train: ', X_train.shape,  'y_train: ', y_train.shape)
#print('X_validation', X_valid.shape, 'y_validation: ', y_valid.shape)
print('X_test', X_test.shape, 'y_test: ', y_test.shape)


# check the proportions
total = X_train.shape[0]  + X_test.shape[0]
print('X_train proportion:', X_train.shape[0] / total)
#print('X_validation proportion:', X_valid.shape[0] / total)
print('X_test proportion:', X_test.shape[0] / total)

### The Model

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
#dvalidation = xgb.DMatrix(X_valid, label=y_valid)
dtest = xgb.DMatrix(X_test)

## gbtree
# instantiate params
params = {}

# general params
general_params = {'silent': 1, 'booster' : 'gbtree'}
params.update(general_params)

# booster params
n_gpus = 3
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['n_gpus'] = n_gpus
params.update(booster_params)

# learning task params
#Check XGBoost manual for full parameter descriptions
learning_task_params = {'eval_metric': 'rmse', 'objective': 'reg:linear'}


params.update(learning_task_params)
print(params)

from xgboost import cv

xgb_cv = cv(dtrain = dtrain, params = params, nfold = 10, early_stopping_rounds = 10, metrics = "rmse")
xgb_cv




Model training settings
Could set rounds to 10,000 with an early stopper. If no change in 25 loops can stop and return to best iteration

In [ ]:
#evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
#watchlist = [(dvalidation,'valid'), (dtrain, 'train')]
watchlist = [ (dtrain, 'train')]

#evals_result = {'valid':{}, 'train':{}}
evals_result = {'train':{}}

num_round = 208
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

#Train model
bst = xgb.train(params,
               dtrain,
               num_round,
               watchlist,
               evals_result = evals_result,
               early_stopping_rounds = 15,
               verbose_eval = 100
               )


### Testing Model Performance

In [ ]:
preds = bst.predict(dtest, ntree_limit =  bst.best_ntree_limit)
#preds
import scipy as scipy
slope, intercept, r, p, stderr = scipy.stats.linregress(y_test, preds)
line = f'Regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
line
'Regression line: y=18.78+0.58x, r=0.73'
fig, ax = plt.subplots()
ax.plot(y_test,preds, linewidth=0, marker='s', label='Data points', alpha = 0.15)
ax.plot(y_test, intercept + slope * y_test, label=line)

ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.legend(facecolor='white')
plt.show()
#plt.figure(figsize=(8,6))
fig, ax = plt.subplots()


ax.hist(y_test, bins=100, alpha=0.3, label="Actual", color = "green")
ax.hist(preds, bins=100, alpha=0.35, label="Predicted", color = "purple")

ax.set_xlabel("Feed intake (kg/day)", size=14)
ax.set_ylabel("Frequency", size=14)
ax.legend(facecolor='white')

plt.title("Distribution of Actual vs Predicted Feed Intake (FI-0 PM)")
plt.savefig('XGBDist2.png', dpi=250)

#plt.savefig("overlapping_histograms_with_matplotlib_Python.png")
plt.figure(figsize=(8,6))
plt.hist(y_test, bins=100, alpha=0.3, label="Actual", color = "green")
plt.hist(preds, bins=100, alpha=0.35, label="Predicted", color = "purple")

plt.xlabel("Feed intake (kg/day)", size=14)
plt.ylabel("Frequency", size=14)
plt.title("Distribution of Actual vs Predicted ")
plt.legend(loc='upper right')
#plt.savefig("overlapping_histograms_with_matplotlib_Python.png")

y_test.shape
preds.shape
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, p = pearsonr(y_test, preds)
print('Pearsons correlation: %.3f' % corr)
from sklearn.metrics import mean_squared_error, r2_score
# calculate coeff of determination
r2 = r2_score(y_test, preds)
print('Coeff of determination: %.3f' % r2)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print('RMSE of prediction: %.3f' % rmse)
from xgboost import cv

r,p = pearsonr(preds, y_test)
import scipy as sp
m_slope, c_intercept, r_value, p_value, std_err = sp.stats.linregress(y_test, preds)
m, c = np.polyfit(y_test, preds, 1
                 )
std_err
xgb.plot_importance("bst")
xgb.plot_importance("bst", importance_type = "cover")
xgb.plot_importance("bst", importance_type = "gain")


### Data Export
Create dataframe with just actual values, predicted values and EAR_TAG. this will be used for genetic analysis of phenotype.

In [ ]:
export_data = {'Actual': y_test,
              'Preds': preds 
              }
df = pd.DataFrame(export_data, columns = ['Actual', 'Preds'])
print(df)